In [ ]:
!pip install openai
!pip install langchain
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 7.9 MB/s e

In [ ]:
#環境設定
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "My_key"

In [ ]:
# 下載外部程式碼以及資料
!git clone https://github.com/u8621011/ironman2023.git

import sys
sys.path.append("/content/ironman2023")

Cloning into 'ironman2023'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 97 (delta 34), reused 81 (delta 18), pack-reused 0
Receiving objects: 100% (97/97), 3.87 MiB | 21.55 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI

# 系統設定
model_name = 'gpt-3.5-turbo'
temperature = 0

# 使用者設定
user_lang = "中文"
learning_lang = "英文"

# 外部資料 - 使用 TextLoader 讀取 txt 檔案
file_name = '/content/ironman2023/text_data/World Stories to Help You Learn _ practice English with Spotlight - English.txt'
loader = TextLoader(file_name, encoding="utf-8")
documents = loader.load()

print(documents)
len(documents[0].page_content)

chat_model = ChatOpenAI(model_name=model_name, temperature=temperature)

[Document(page_content='Welcome to Spotlight. I’m Adam Navis.\nSpotlight uses a special English method of broadcasting.\nIt is easier for people to understand, no matter where in the world they live.\nWe begin today’s program with a story.\nKwesuka sukela - once upon a time - there lived a woman named Manzandaba and her husband Zenzele.\nThey lived in a traditional home in a small Zulu village.\nAt night they sat around their open fire with their many children.\n“Mama” the children cried. "Sifuna izindaba! We want stories!\nTell us some stories, mother!”\nManzandaba would think and think.\nBut she and Zenzele did not know any stories.\nThey asked their neighbors, but their neighbors did not know any stories.\nThey listened to the wind.\nBut the wind did not know any stories.\nThere were no stories anywhere.\nCan you imagine a world without stories?\nThis story we just began is a story from the Zulu people of Southern Africa.\nIt is a well-known imaginary story.\nIt tells how all storie

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
line_data = documents[0].page_content.splitlines()

# 然後隨機挑選 20 句例句來模擬挑選出來的教學例句，
import random
random.seed(0)
recommend_lines = random.sample(line_data, 20)

print(recommend_lines)

['The two men argued over and over.', 'Now you have started a war between us!”', 'But she and Zenzele did not know any stories.', 'This time, no one laughed at all.', 'This program is called “World Stories to Help You Learn”.', 'You can also find us on Facebook and YouTube.', 'He listened to their shouts.“Our friendship is over!”', 'When they got older, they built their houses opposite each other.', 'You can leave a comment on our website.', '“No. His coat was red.', 'But some storytellers use stories to teach good advice or give a moral lesson.', 'You can listen to this program again, and read it, on the internet at www.spotlightenglish.com.', 'Finally, a big sea turtle offered to take Manzandaba to the Spirit People under the sea.', 'Our final story also has a moral meaning behind it.', 'So today, we will celebrate this day with you!', 'A few of the people smiled.', 'The Spirit People agreed to give Manzandaba stories if', 'One day a trickster came from the village.', 'Today’s Spotli

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template = """你是一個專業的外語老師，你有一個特殊專長是在能夠以影片的內容找出值得教學的內容，你也很擅長做課程的規劃。

接下來我會提供給挑選出來的教學例句，請在所有教學例句裏面隨機找出 10 個左右值得教學的詞彙（教學詞彙）。
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "教學例句：\n{captions}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_messages = chat_prompt.format_messages(captions=recommend_lines)
chat_model(chat_messages)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='教學詞彙：\n1. argued\n2. war\n3. stories\n4. laughed\n5. program\n6. Facebook\n7. YouTube\n8. friendship\n9. houses\n10. comment\n11. coat\n12. storytellers\n13. moral\n14. internet\n15. sea turtle\n16. celebrate\n17. smiled\n18. trickster\n19. Spotlight\n20. began')

In [ ]:
# 使用 CommaSeparatedListOutputParser 生成格式指令
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

print(output_parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [ ]:
# 將教學詞彙推薦的提示訊息加上 OutputParser 的指令
system_template = """你是一個專業的外語老師，你有一個特殊專長是在能夠以影片的內容找出值得教學的內容，你也很擅長做課程的規劃。

接下來我會提供給挑選出來的教學例句，請在所有教學例句裏面隨機找出 10 個左右值得教學的詞彙（教學詞彙）。
{format_instructions}
"""

system_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        template = system_template,
        input_variables=[],
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )
)

human_template = "教學例句：\n{captions}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_messages = chat_prompt.format_messages(captions=recommend_lines)
response = chat_model(chat_messages)

print(response)

content='argued, war, stories, laughed, program, Facebook, YouTube, shouts, houses, comment, coat, storytellers, teach, advice, moral, celebrate, smiled, agreed, trickster, Spotlight'


In [ ]:
output_parser.parse(response.content)

['argued',
 'war',
 'stories',
 'laughed',
 'program',
 'Facebook',
 'YouTube',
 'shouts',
 'houses',
 'comment',
 'coat',
 'storytellers',
 'teach',
 'advice',
 'moral',
 'celebrate',
 'smiled',
 'agreed',
 'trickster',
 'Spotlight']